## Libraries 

In [1]:
import pandas as pd
import numpy as np
import os
# change the working directory to the parent directory
current_dir = os.path.abspath(os.getcwd())

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)

## Loading data from the eICU dataset

Preprocessing can be performed in the same way for data intended for 12-, 24-, and 48-hour predictions. The data cleaning and transformation steps are identical in each case, meaning that only the file paths need to be adjusted for each time-based dataset. Once the paths are updated, the preprocessing process automatically adapts without requiring additional modifications, ensuring an efficient and consistent workflow.

In [2]:
data_test = pd.read_csv("Data_johan/eicu/48h_24h/demograficos.csv" )
data_test1 = pd.read_csv("Data_johan/eicu/48h_24h/vitalsigns.csv" )
data_test2 = pd.read_csv("Data_johan/eicu/48h_24h/lab.csv" )
data_test3 = pd.read_csv("Data_johan/eicu/48h_24h/vasopresores.csv" )
data_test4 = pd.read_csv("Data_johan/eicu/48h_24h/antibioticos.csv" )
data_test5 = pd.read_csv("Data_johan/eicu/48h_24h/gcs.csv" )
data_test6 = pd.read_csv("Data_johan/eicu/48h_24h/urineoutput.csv" )
data_test7 = pd.read_csv("Data_johan/eicu/48h_24h/sofa.csv" )
data_test8 = pd.read_csv('datapre/eicu/conflag_eICU_48_72.csv')

## Reviewing demographic data from the eICU dataset

In [3]:
print(data_test.head())

   patientunitstayid age  gender  ethnicity  icu_los_hours  \
0             224606  76       0  Caucasian             69   
1             151900  66       0  Caucasian             57   
2             197921  67       0  Caucasian             91   
3             145917  53       1      Asian             77   
4             210208  52       1      Asian             91   

   hospital_expire_flag  
0                     0  
1                     0  
2                     0  
3                     0  
4                     0  


In [4]:
print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   patientunitstayid     5553 non-null   int64 
 1   age                   5553 non-null   object
 2   gender                5553 non-null   int64 
 3   ethnicity             5521 non-null   object
 4   icu_los_hours         5553 non-null   int64 
 5   hospital_expire_flag  5553 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 260.4+ KB
None


In [5]:
print(data_test.describe())

       patientunitstayid       gender  icu_los_hours  hospital_expire_flag
count       5.553000e+03  5553.000000    5553.000000           5553.000000
mean        1.736428e+06     0.510895     145.492346              0.110751
std         1.040834e+06     0.499926     207.482509              0.313852
min         1.413040e+05     0.000000      48.000000              0.000000
25%         7.549710e+05     0.000000      67.000000              0.000000
50%         1.710390e+06     1.000000      94.000000              0.000000
75%         2.791676e+06     1.000000     165.000000              0.000000
max         3.352922e+06     1.000000    8299.000000              1.000000


## Data treatment for the age column

In [6]:
# Filter values in 'age' that cannot be converted to numbers
problematic_values = data_test[~data_test['age'].str.isnumeric()]
print(problematic_values['age'])


8       > 89
10      > 89
33      > 89
35      > 89
41      > 89
        ... 
5306    > 89
5438    > 89
5505    > 89
5508    > 89
5546    > 89
Name: age, Length: 251, dtype: object


In [7]:
# Replace values '> 89' with 90 in the 'age' column
data_test['age'] = data_test['age'].replace('> 89', '90')

# Convert the 'age' column to a numeric type
data_test['age'] = pd.to_numeric(data_test['age'], errors='coerce')



In [8]:
print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   patientunitstayid     5553 non-null   int64 
 1   age                   5553 non-null   int64 
 2   gender                5553 non-null   int64 
 3   ethnicity             5521 non-null   object
 4   icu_los_hours         5553 non-null   int64 
 5   hospital_expire_flag  5553 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 260.4+ KB
None


## Data treatment for the ethnicity column

In [9]:
# Display unique values in the 'ethnicity' column
print(data_test['ethnicity'].unique())



['Caucasian' 'Asian' 'African American' 'Hispanic' 'Native American' nan
 'Other/Unknown']


In [10]:
# Define a mapping for the ethnicity values
ethnicity_mapping = {
    'Caucasian': 0,
    'Asian': 1,
    'African American': 2,
    'Hispanic': 3,
    'Native American': 4,
    'Other/Unknown': 5,
    np.nan: 5  # Replace NaN with 5
}

# Replace values in the 'ethnicity' column using the mapping
data_test['ethnicity'] = data_test['ethnicity'].replace(ethnicity_mapping)

C:\Users\JOHAN\AppData\Local\Temp\ipykernel_2076\3784381758.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_test['ethnicity'] = data_test['ethnicity'].replace(ethnicity_mapping)


In [11]:
print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patientunitstayid     5553 non-null   int64  
 1   age                   5553 non-null   int64  
 2   gender                5553 non-null   int64  
 3   ethnicity             5553 non-null   float64
 4   icu_los_hours         5553 non-null   int64  
 5   hospital_expire_flag  5553 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 260.4 KB
None


## Renaming columns and adjusting data types

In [12]:
# Renaming columns
data_test = data_test.rename(columns={
    'gender': 'sex',
    'ethnicity': 'race',
    'icu_los_hours': 'los_icu'
})

# Changing the data types
data_test['sex'] = data_test['sex'].astype('int64')  # Ensure 'sex' is of type int64
data_test['race'] = data_test['race'].astype('int64')  # Ensure 'race' is of type int64
data_test['los_icu'] = data_test['los_icu'].astype('float64')  # Ensure 'los_icu' is of type float64
data_test['los_icu'] = data_test['los_icu'] / 24


In [14]:
data_test = data_test.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patientunitstayid     3786 non-null   int64  
 1   age                   3786 non-null   int64  
 2   sex                   3786 non-null   int64  
 3   race                  3786 non-null   int64  
 4   los_icu               3786 non-null   float64
 5   hospital_expire_flag  3786 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 177.6 KB
None


## Reviewing vital signs data from the eICU dataset

In [16]:
print(data_test1.head())

   patientunitstayid  max_temperature  min_temperature  avg_temperature  \
0             141304             36.7             36.1        36.483333   
1             141751             37.6             36.3        36.803497   
2             141920             37.7             36.6        37.133333   
3             141945             36.7             36.5        36.660000   
4             141959             37.2             36.1        36.620000   

   max_spo2  min_spo2   avg_spo2  max_heartrate  min_heartrate  avg_heartrate  \
0      97.0      83.0  93.740385          114.0           76.0      84.834951   
1     100.0      96.0  98.710345          119.0           81.0      93.031579   
2     100.0      90.0  96.721713          135.0           66.0      87.330275   
3     100.0      79.0  96.021277           96.0           69.0      78.905229   
4     100.0      91.0  97.628866           98.0           58.0      75.941581   

   ...  avg_resp_rate  max_sbp  min_sbp     avg_sbp  max_dbp  

In [19]:
print(data_test1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  5553 non-null   int64  
 1   max_temperature    5079 non-null   float64
 2   min_temperature    5079 non-null   float64
 3   avg_temperature    5079 non-null   float64
 4   max_spo2           5526 non-null   float64
 5   min_spo2           5526 non-null   float64
 6   avg_spo2           5526 non-null   float64
 7   max_heartrate      5537 non-null   float64
 8   min_heartrate      5537 non-null   float64
 9   avg_heartrate      5537 non-null   float64
 10  max_resp_rate      5488 non-null   float64
 11  min_resp_rate      5488 non-null   float64
 12  avg_resp_rate      5488 non-null   float64
 13  max_sbp            5009 non-null   float64
 14  min_sbp            5009 non-null   float64
 15  avg_sbp            5009 non-null   float64
 16  max_dbp            5009 

In [18]:
print(data_test1.describe())

       patientunitstayid  max_temperature  min_temperature  avg_temperature  \
count       5.553000e+03      5079.000000      5079.000000      5079.000000   
mean        1.736428e+06        37.332704        36.241549        36.883687   
std         1.040834e+06         0.747550         1.807506         0.631605   
min         1.413040e+05        22.300000         1.200000        21.730588   
25%         7.549710e+05        36.900000        36.200000        36.576000   
50%         1.710390e+06        37.200000        36.500000        36.834014   
75%         2.791676e+06        37.700000        36.800000        37.180000   
max         3.352922e+06        41.200000        39.400000        39.605556   

          max_spo2     min_spo2     avg_spo2  max_heartrate  min_heartrate  \
count  5526.000000  5526.000000  5526.000000    5537.000000    5537.000000   
mean     99.600253    84.979370    96.601417     112.478418      73.290410   
std       0.982375    12.308461     2.410420      21.6

## Renaming columns 

In [20]:
# Define a mapping for the column renaming
column_mapping = {
    'min_heartrate': 'heart_rate_min',
    'max_heartrate': 'heart_rate_max',
    'avg_heartrate': 'heart_rate_mean',
    'min_sbp': 'sbp_min',
    'max_sbp': 'sbp_max',
    'avg_sbp': 'sbp_mean',
    'min_dbp': 'dbp_min',
    'max_dbp': 'dbp_max',
    'avg_dbp': 'dbp_mean',
    'min_mbp': 'mbp_min',
    'max_mbp': 'mbp_max',
    'avg_mbp': 'mbp_mean',
    'min_resp_rate': 'resp_rate_min',
    'max_resp_rate': 'resp_rate_max',
    'avg_resp_rate': 'resp_rate_mean',
    'min_temperature': 'temperature_min',
    'max_temperature': 'temperature_max',
    'avg_temperature': 'temperature_mean',
    'min_spo2': 'spo2_min',
    'max_spo2': 'spo2_max',
    'avg_spo2': 'spo2_mean'
}

# Rename the columns
data_test1 = data_test1.rename(columns=column_mapping)



In [21]:
data_test1 = data_test1.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   temperature_max    3429 non-null   float64
 2   temperature_min    3429 non-null   float64
 3   temperature_mean   3429 non-null   float64
 4   spo2_max           3770 non-null   float64
 5   spo2_min           3770 non-null   float64
 6   spo2_mean          3770 non-null   float64
 7   heart_rate_max     3772 non-null   float64
 8   heart_rate_min     3772 non-null   float64
 9   heart_rate_mean    3772 non-null   float64
 10  resp_rate_max      3741 non-null   float64
 11  resp_rate_min      3741 non-null   float64
 12  resp_rate_mean     3741 non-null   float64
 13  sbp_max            3401 non-null   float64
 14  sbp_min            3401 non-null   float64
 15  sbp_mean           3401 non-null   float64
 16  dbp_max            3401 

## Removing columns with less than 80% non-null values in a DataFrame

In [22]:
# Calculate the threshold for non-null values
th = 0.80 * len(data_test1)
# Remove columns that have fewer than 'threshold' non-null values
data_test1 = data_test1.loc[:, data_test1.notnull().sum() >= th]
print(data_test1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   temperature_max    3429 non-null   float64
 2   temperature_min    3429 non-null   float64
 3   temperature_mean   3429 non-null   float64
 4   spo2_max           3770 non-null   float64
 5   spo2_min           3770 non-null   float64
 6   spo2_mean          3770 non-null   float64
 7   heart_rate_max     3772 non-null   float64
 8   heart_rate_min     3772 non-null   float64
 9   heart_rate_mean    3772 non-null   float64
 10  resp_rate_max      3741 non-null   float64
 11  resp_rate_min      3741 non-null   float64
 12  resp_rate_mean     3741 non-null   float64
 13  sbp_max            3401 non-null   float64
 14  sbp_min            3401 non-null   float64
 15  sbp_mean           3401 non-null   float64
 16  dbp_max            3401 

## Reviewing laboratory data from the eICU dataset

In [23]:
print(data_test2.head())

   patientunitstayid  aniongap_min  aniongap_max  albumin_min  albumin_max  \
0             141304          17.0          17.0          NaN          NaN   
1             141751          13.0          17.0          1.7          1.7   
2             141920          14.0          14.0          2.7          2.7   
3             141945          10.0          10.0          NaN          NaN   
4             141959          10.0          10.0          NaN          NaN   

   bands_min  bands_max  bicarbonate_min  bicarbonate_max  hco3_min  ...  \
0        NaN        NaN             26.0             26.0       NaN  ...   
1        NaN        NaN             18.0             21.0       NaN  ...   
2        NaN        NaN             24.0             24.0       NaN  ...   
3        NaN        NaN             31.0             31.0       NaN  ...   
4        1.0        1.0             26.0             26.0      22.0  ...   

   pt_min  pt_max  sodium_min  sodium_max  bun_min  bun_max  wbc_min  wbc_

In [24]:
print(data_test2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  5553 non-null   int64  
 1   aniongap_min       4054 non-null   float64
 2   aniongap_max       4054 non-null   float64
 3   albumin_min        2863 non-null   float64
 4   albumin_max        2863 non-null   float64
 5   bands_min          500 non-null    float64
 6   bands_max          500 non-null    float64
 7   bicarbonate_min    4897 non-null   float64
 8   bicarbonate_max    4897 non-null   float64
 9   hco3_min           1497 non-null   float64
 10  hco3_max           1497 non-null   float64
 11  bilirubin_min      2522 non-null   float64
 12  bilirubin_max      2522 non-null   float64
 13  creatinine_min     5200 non-null   float64
 14  creatinine_max     5200 non-null   float64
 15  chloride_min       5206 non-null   float64
 16  chloride_max       5206 

In [25]:
print(data_test2.describe())

       patientunitstayid  aniongap_min  aniongap_max  albumin_min  \
count       5.553000e+03   4054.000000   4054.000000  2863.000000   
mean        1.736428e+06     10.075925     10.550493     2.323577   
std         1.040834e+06      4.162147      4.446670     0.589163   
min         1.413040e+05      0.500000      0.500000     0.800000   
25%         7.549710e+05      7.000000      7.300000     1.900000   
50%         1.710390e+06     10.000000     10.000000     2.300000   
75%         2.791676e+06     12.400000     13.000000     2.700000   
max         3.352922e+06     37.000000     48.000000     4.600000   

       albumin_max   bands_min   bands_max  bicarbonate_min  bicarbonate_max  \
count  2863.000000  500.000000  500.000000      4897.000000      4897.000000   
mean      2.343591    9.709380   10.441380        24.276455        24.769308   
std       0.596671   10.489606   11.450574         5.417585         5.180336   
min       0.800000    1.000000    1.000000         2.90000

In [26]:
data_test2 = data_test2.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   aniongap_min       2752 non-null   float64
 2   aniongap_max       2752 non-null   float64
 3   albumin_min        1884 non-null   float64
 4   albumin_max        1884 non-null   float64
 5   bands_min          303 non-null    float64
 6   bands_max          303 non-null    float64
 7   bicarbonate_min    3307 non-null   float64
 8   bicarbonate_max    3307 non-null   float64
 9   hco3_min           1027 non-null   float64
 10  hco3_max           1027 non-null   float64
 11  bilirubin_min      1642 non-null   float64
 12  bilirubin_max      1642 non-null   float64
 13  creatinine_min     3511 non-null   float64
 14  creatinine_max     3511 non-null   float64
 15  chloride_min       3519 non-null   float64
 16  chloride_max       3519 

## Removing columns with less than 80% non-null values in a DataFrame

In [27]:
# Calculate the threshold for non-null values
th = 0.80 * len(data_test2)
# Remove columns that have fewer than 'threshold' non-null values
data_test2 = data_test2.loc[:, data_test2.notnull().sum() >= th]
print(data_test2.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   bicarbonate_min    3307 non-null   float64
 2   bicarbonate_max    3307 non-null   float64
 3   creatinine_min     3511 non-null   float64
 4   creatinine_max     3511 non-null   float64
 5   chloride_min       3519 non-null   float64
 6   chloride_max       3519 non-null   float64
 7   glucose_min        3488 non-null   float64
 8   glucose_max        3488 non-null   float64
 9   hematocrit_min     3392 non-null   float64
 10  hematocrit_max     3392 non-null   float64
 11  hemoglobin_min     3395 non-null   float64
 12  hemoglobin_max     3395 non-null   float64
 13  platelet_min       3366 non-null   float64
 14  platelet_max       3366 non-null   float64
 15  potassium_min      3560 non-null   float64
 16  potassium_max      3560 

## Reviewing vasopressors data from the eICU dataset

In [28]:
print(data_test3.head())

   patientunitstayid  norepinephrine  epinephrine  dopamine  dobutamine  \
0            1580187             0.0          0.0       0.0         0.0   
1             617217             0.0          1.0       0.0         0.0   
2            3341184             0.0          0.0       0.0         0.0   
3            3024088             0.0          0.0       0.0         0.0   
4            1769940             0.0          0.0       0.0         0.0   

   phenylephrine  vasopressin  milrinone  heparin  warfarin  
0            0.0          0.0        0.0      0.0       0.0  
1            1.0          0.0        0.0      0.0       0.0  
2            0.0          0.0        0.0      0.0       0.0  
3            0.0          0.0        0.0      0.0       0.0  
4            0.0          0.0        0.0      0.0       0.0  


In [29]:
print(data_test3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  5553 non-null   int64  
 1   norepinephrine     3626 non-null   float64
 2   epinephrine        3626 non-null   float64
 3   dopamine           3626 non-null   float64
 4   dobutamine         3626 non-null   float64
 5   phenylephrine      3626 non-null   float64
 6   vasopressin        3626 non-null   float64
 7   milrinone          3626 non-null   float64
 8   heparin            3626 non-null   float64
 9   warfarin           3626 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 434.0 KB
None


In [30]:
print(data_test3.describe())

       patientunitstayid  norepinephrine  epinephrine     dopamine  \
count       5.553000e+03     3626.000000  3626.000000  3626.000000   
mean        1.736428e+06        0.047159     0.008274     0.002206   
std         1.040834e+06        0.249101     0.119484     0.062106   
min         1.413040e+05        0.000000     0.000000     0.000000   
25%         7.549710e+05        0.000000     0.000000     0.000000   
50%         1.710390e+06        0.000000     0.000000     0.000000   
75%         2.791676e+06        0.000000     0.000000     0.000000   
max         3.352922e+06        5.000000     4.000000     2.000000   

        dobutamine  phenylephrine  vasopressin  milrinone      heparin  \
count  3626.000000    3626.000000  3626.000000     3626.0  3626.000000   
mean      0.001931       0.018478     0.015996        0.0     0.084115   
std       0.043901       0.169186     0.129798        0.0     0.326040   
min       0.000000       0.000000     0.000000        0.0     0.000000   

In [31]:
data_test3 = data_test3.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   norepinephrine     2370 non-null   float64
 2   epinephrine        2370 non-null   float64
 3   dopamine           2370 non-null   float64
 4   dobutamine         2370 non-null   float64
 5   phenylephrine      2370 non-null   float64
 6   vasopressin        2370 non-null   float64
 7   milrinone          2370 non-null   float64
 8   heparin            2370 non-null   float64
 9   warfarin           2370 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 295.9 KB
None


## Removing columns 

In [32]:
data_test3 = data_test3.drop(columns=['heparin'])
data_test3 = data_test3.drop(columns=['warfarin'])
data_test3.fillna(0, inplace=True)
print(data_test3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   norepinephrine     3786 non-null   float64
 2   epinephrine        3786 non-null   float64
 3   dopamine           3786 non-null   float64
 4   dobutamine         3786 non-null   float64
 5   phenylephrine      3786 non-null   float64
 6   vasopressin        3786 non-null   float64
 7   milrinone          3786 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 236.8 KB
None


## Reviewing antibiotics data from the eICU dataset

In [33]:
print(data_test4.head())

   patientunitstayid  Grupo Cefalosporinas  Grupo Penicilinas  Macrolides  \
0             141304                     0                  0           0   
1             141751                     0                  1           0   
2             141920                     0                  0           0   
3             141945                     0                  0           0   
4             141959                     0                  0           0   

   Meropenem  Metronidazole  Quinolonas  Vancomycin  other  
0          0              0           0           1      0  
1          0              0           0           1      0  
2          0              0           0           0      1  
3          0              0           0           0      1  
4          0              0           0           0      1  


In [34]:
print(data_test4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   patientunitstayid     5553 non-null   int64
 1   Grupo Cefalosporinas  5553 non-null   int64
 2   Grupo Penicilinas     5553 non-null   int64
 3   Macrolides            5553 non-null   int64
 4   Meropenem             5553 non-null   int64
 5   Metronidazole         5553 non-null   int64
 6   Quinolonas            5553 non-null   int64
 7   Vancomycin            5553 non-null   int64
 8   other                 5553 non-null   int64
dtypes: int64(9)
memory usage: 390.6 KB
None


In [35]:
print(data_test4.describe())

       patientunitstayid  Grupo Cefalosporinas  Grupo Penicilinas  \
count       5.553000e+03           5553.000000        5553.000000   
mean        1.736428e+06              0.013326           0.020890   
std         1.040834e+06              0.122279           0.149191   
min         1.413040e+05              0.000000           0.000000   
25%         7.549710e+05              0.000000           0.000000   
50%         1.710390e+06              0.000000           0.000000   
75%         2.791676e+06              0.000000           0.000000   
max         3.352922e+06              2.000000           3.000000   

        Macrolides    Meropenem  Metronidazole   Quinolonas   Vancomycin  \
count  5553.000000  5553.000000    5553.000000  5553.000000  5553.000000   
mean      0.003061     0.005943       0.006843     0.018188     0.084459   
std       0.055250     0.083601       0.082447     0.144023     0.312270   
min       0.000000     0.000000       0.000000     0.000000     0.000000  

## Renaming columns and adjusting data types

In [36]:
data_test4 = data_test4.drop(columns=['other'])
# Renaming columns
data_test4 = data_test4.rename(columns={
    'Grupo Cefalosporinas': 'Cefalosporine',
    'Grupo Penicilinas': 'Penicillin',
    'Macrolides': 'Macrolide',
    'Quinolonas' : 'Quinolone',
})

# Converting specified columns to float64
data_test4[['Cefalosporine', 'Penicillin', 'Macrolide', 'Quinolone', 'Meropenem', 'Metronidazole', 'Vancomycin']] = data_test4[['Cefalosporine', 'Penicillin', 'Macrolide', 'Quinolone', 'Meropenem', 'Metronidazole', 'Vancomycin']].astype('float64')



In [37]:
data_test4 = data_test4.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   Cefalosporine      3786 non-null   float64
 2   Penicillin         3786 non-null   float64
 3   Macrolide          3786 non-null   float64
 4   Meropenem          3786 non-null   float64
 5   Metronidazole      3786 non-null   float64
 6   Quinolone          3786 non-null   float64
 7   Vancomycin         3786 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 236.8 KB
None


## Reviewing Glasgow Coma Scale (GCS) data from the eICU dataset

In [38]:
print(data_test5.head())

   patientunitstayid   gcs  gcsmotor  gcsverbal  gcseyes
0             141304  15.0       NaN        NaN      NaN
1             141751  10.0       NaN        NaN      NaN
2             141920  15.0       NaN        NaN      NaN
3             141945  15.0       NaN        NaN      NaN
4             141959  14.0       NaN        NaN      NaN


In [39]:
print(data_test5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  5553 non-null   int64  
 1   gcs                4126 non-null   float64
 2   gcsmotor           3007 non-null   float64
 3   gcsverbal          2994 non-null   float64
 4   gcseyes            3007 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 217.0 KB
None


In [40]:
print(data_test5.describe())

       patientunitstayid          gcs     gcsmotor    gcsverbal      gcseyes
count       5.553000e+03  4126.000000  3007.000000  2994.000000  3007.000000
mean        1.736428e+06    13.328648     5.701696     3.923848     3.779514
std         1.040834e+06     2.662892     0.853045     1.563089     0.620888
min         1.413040e+05     3.000000     1.000000     1.000000     1.000000
25%         7.549710e+05    12.000000     6.000000     3.000000     4.000000
50%         1.710390e+06    15.000000     6.000000     5.000000     4.000000
75%         2.791676e+06    15.000000     6.000000     5.000000     4.000000
max         3.352922e+06    15.000000     6.000000     5.000000     4.000000


## Renaming columns 

In [41]:
# Renaming columns
data_test5 = data_test5.rename(columns={
    'gcs': 'gcs_min',
    'gcsmotor': 'gcs_motor',
    'gcsverbal' : 'gcs_verbal',
    'gcseyes' : 'gcs_eyes'
})

In [42]:
data_test5 = data_test5.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   gcs_min            2827 non-null   float64
 2   gcs_motor          2036 non-null   float64
 3   gcs_verbal         2025 non-null   float64
 4   gcs_eyes           2036 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 148.0 KB
None


In [51]:
data_test6 = data_test6.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test6.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  3786 non-null   int64  
 1   urineoutput        2164 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 59.3 KB
None


In [55]:
data_test7 = data_test7.merge(data_test8[['patientunitstayid']], on=['patientunitstayid'], how='inner')
print(data_test7.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   patientunitstayid  3786 non-null   int64
 1   sofa               3786 non-null   int64
dtypes: int64(2)
memory usage: 59.3 KB
None


In [56]:
# Start with df1, which contains patientunitstayid and hospital_expire_flag columns
# Ensure 'df1' has only 'patientunitstayid' and 'hospital_expire_flag' columns
merged_df = data_test[['patientunitstayid', 'hospital_expire_flag', 'age', 'sex', 'race', 'los_icu']]

# Merge df2 to df8 sequentially with 'merged_df' using an inner join on 'patientunitstayid'
# This will only keep rows with matching 'patientunitstayid' values across all dataframes

for df in [data_test1, data_test2, data_test3, data_test4, data_test5, data_test6, data_test7]:
    merged_df = merged_df.merge(df, on='patientunitstayid', how='inner')



In [57]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 71 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patientunitstayid     3786 non-null   int64  
 1   hospital_expire_flag  3786 non-null   int64  
 2   age                   3786 non-null   int64  
 3   sex                   3786 non-null   int64  
 4   race                  3786 non-null   int64  
 5   los_icu               3786 non-null   float64
 6   temperature_max       3429 non-null   float64
 7   temperature_min       3429 non-null   float64
 8   temperature_mean      3429 non-null   float64
 9   spo2_max              3770 non-null   float64
 10  spo2_min              3770 non-null   float64
 11  spo2_mean             3770 non-null   float64
 12  heart_rate_max        3772 non-null   float64
 13  heart_rate_min        3772 non-null   float64
 14  heart_rate_mean       3772 non-null   float64
 15  resp_rate_max        

In [58]:
# Check for columns that contain only 0 values
columns_with_zeros = merged_df.columns[(merged_df == 0).all()]
print(columns_with_zeros)

# Drop the columns that contain only zeros
merged_df = merged_df.drop(columns=columns_with_zeros)
print(merged_df.info())

Index(['milrinone'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 70 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patientunitstayid     3786 non-null   int64  
 1   hospital_expire_flag  3786 non-null   int64  
 2   age                   3786 non-null   int64  
 3   sex                   3786 non-null   int64  
 4   race                  3786 non-null   int64  
 5   los_icu               3786 non-null   float64
 6   temperature_max       3429 non-null   float64
 7   temperature_min       3429 non-null   float64
 8   temperature_mean      3429 non-null   float64
 9   spo2_max              3770 non-null   float64
 10  spo2_min              3770 non-null   float64
 11  spo2_mean             3770 non-null   float64
 12  heart_rate_max        3772 non-null   float64
 13  heart_rate_min        3772 non-null   float64
 14  heart_rate_mean       3772 non-null

In [59]:
merged_df.to_csv('datapre/eicu/conflag_eICU_24_48.csv', index=False)

In [60]:
merged_df = merged_df.drop(columns=['patientunitstayid', 'hospital_expire_flag'])
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 68 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               3786 non-null   int64  
 1   sex               3786 non-null   int64  
 2   race              3786 non-null   int64  
 3   los_icu           3786 non-null   float64
 4   temperature_max   3429 non-null   float64
 5   temperature_min   3429 non-null   float64
 6   temperature_mean  3429 non-null   float64
 7   spo2_max          3770 non-null   float64
 8   spo2_min          3770 non-null   float64
 9   spo2_mean         3770 non-null   float64
 10  heart_rate_max    3772 non-null   float64
 11  heart_rate_min    3772 non-null   float64
 12  heart_rate_mean   3772 non-null   float64
 13  resp_rate_max     3741 non-null   float64
 14  resp_rate_min     3741 non-null   float64
 15  resp_rate_mean    3741 non-null   float64
 16  sbp_max           3401 non-null   float64


In [61]:
merged_df.to_csv('datapre/eicu/sinflag_eICU_24_48.csv', index=False)